# Tugas Scraping PTA

In [ ]:
 _= !pip install bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from google.colab import files
_= !pip install Sastrawi

In [ ]:
def pta ():
  data= {'judul':[] ,'penulis' :[],'dp1':[], 'dp2':[], 'abstrak':[]}

  for i in range (1,172):
    url = "https://pta.trunojoyo.ac.id/c_search/byprod/10/{}".format(i)
    r = requests.get(url)
    request = r.content
    soup = BeautifulSoup(request,"html.parser")
    artikels = soup.findAll('li', attrs={'data-cat':'#luxury'})
    for artikel in artikels:
      link = artikel.find('a', class_='gray button')['href']
      respone = requests.get(link)
      soup_data = BeautifulSoup(respone.content, 'html.parser')
      cont = soup_data.find('div', attrs={'id':'content_journal'})
      info = cont.findAll('span')
      judul = cont.find('a', class_='title').get_text()
      pen = info[0].get_text().split(':')[1]
      dp1 = info[1].get_text().split(':')[1]
      dp2 = info[2].get_text().split(':')[1]
      abs = ' '.join(cont.find('p').get_text().splitlines()).capitalize()
      data['judul'].append(judul)
      data['penulis'].append(pen)
      data['dp1'].append(dp1)
      data['dp2'].append(dp2)
      data['abstrak'].append(abs)

  df=pd.DataFrame(data)
  df.to_csv("DataPTA.csv",index=False)
  return df



In [ ]:
pta()

,judul,penulis,dp1,dp2,abstrak
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (siakad) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di perguruan tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...
850,Sistem Peramalan Stok Barang dengan menggunaka...,Devi Editya Susanti,"Sigit Susanto Putro, S.Kom., M.Kom","Ach. Khozaimi, S.Kom.,M.Kom",Toko tas deeolshop menjual berbagai macam tas ...
851,Sistem Rekomendasi Dalam Menentukan Posisi Dep...,Kumayyah,"Muhammad Ali Syakur, S.Si., MT","Sigit Susanto Putro, S.Kom., M.Kom.",Sepak bola adalah salah satu olahraga yang saa...
852,MULTI CRITERIA DECISION MAKING UNTUK MENENTUKA...,Ahmad Faris Ihsan,"Moch. Kautsar Sophan, S.Kom., M.MT.","Dr. Yeni Kustiyahningsih, S.Kom., M.Kom.",Perguruan tinggi merupakan salah satu jenjang ...
853,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Rachmad Agung Pambudi,"Eka Mala Sari Rochman, S.Kom., M.Kom","Sri Herawati, S.Kom., M.Kom",Investasi saham selama ini memiliki resiko ker...


In [ ]:
files.download('DataPTA.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Punctuation process**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
import re, string
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data = pd.read_csv("/content/DataPTA.csv")
print (data)

                                                 judul  \
0    PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...   
1    APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...   
2    RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...   
3    SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...   
4    SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...   
..                                                 ...   
850  Sistem Peramalan Stok Barang dengan menggunaka...   
851  Sistem Rekomendasi Dalam Menentukan Posisi Dep...   
852  MULTI CRITERIA DECISION MAKING UNTUK MENENTUKA...   
853  PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...   
854  SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...   

                    penulis                                    dp1  \
0        A.Ubaidillah S.Kom                       Budi Setyono M.T   
1       M. Basith Ardianto,                  Drs. Budi Soesilo, MT   
2     Akhmad Suyandi, S.Kom                 Drs. Budi Soesilo, M.T   
3           Heri Supriy

In [ ]:
data = pd.DataFrame (data['abstrak'])

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def cleaning(text):
    # HTML Tag Removal
    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [ ]:
data['abstrak'] = data['abstrak'].apply(lambda x: cleaning(x))
data.head()

,abstrak
0,sistem informasi akademik siakad merupakan sis...
1,berjalannya koneksi jaringan komputer dengan l...
2,web server adalah sebuah perangkat lunak serve...
3,penjadwalan kuliah di perguruan tinggi merupak...
4,seiring perkembangan teknologi yang ada diduni...


# **Tokenisasi**

In [ ]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [ ]:
data['abstrak_Tokens'] = data['abstrak'].apply(lambda x: word_tokenize(x))
data[["abstrak", "abstrak_Tokens"]].head()

,abstrak,abstrak_Tokens
0,sistem informasi akademik siakad merupakan sis...,"[sistem, informasi, akademik, siakad, merupaka..."
1,berjalannya koneksi jaringan komputer dengan l...,"[berjalannya, koneksi, jaringan, komputer, den..."
2,web server adalah sebuah perangkat lunak serve...,"[web, server, adalah, sebuah, perangkat, lunak..."
3,penjadwalan kuliah di perguruan tinggi merupak...,"[penjadwalan, kuliah, di, perguruan, tinggi, m..."
4,seiring perkembangan teknologi yang ada diduni...,"[seiring, perkembangan, teknologi, yang, ada, ..."


# **Stopword**

In [ ]:
from nltk.corpus import stopwords
from itertools import chain

stop_words = set(chain(stopwords.words('indonesian'), stopwords.words('english')))

data['abstrak_Tokens'] = data['abstrak_Tokens'].apply(lambda x: [w for w in x if not w in stop_words])

In [ ]:
data[["abstrak", "abstrak_Tokens"]].head()

,abstrak,abstrak_Tokens
0,sistem informasi akademik siakad merupakan sis...,"[sistem, informasi, akademik, siakad, sistem, ..."
1,berjalannya koneksi jaringan komputer dengan l...,"[berjalannya, koneksi, jaringan, komputer, lan..."
2,web server adalah sebuah perangkat lunak serve...,"[web, server, perangkat, lunak, server, berfun..."
3,penjadwalan kuliah di perguruan tinggi merupak...,"[penjadwalan, kuliah, perguruan, kompleks, per..."
4,seiring perkembangan teknologi yang ada diduni...,"[seiring, perkembangan, teknologi, didunia, mu..."
